## Import data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
len(data), data.dtypes

(891,
 PassengerId      int64
 Survived         int64
 Pclass           int64
 Name            object
 Sex             object
 Age            float64
 SibSp            int64
 Parch            int64
 Ticket          object
 Fare           float64
 Cabin           object
 Embarked        object
 dtype: object)

## Convert DataFrame to SQLite Database

In [5]:
from sqlite3 import connect

In [6]:
con = connect('titanic.db')
data.to_sql('Passenger Data', con, if_exists = 'replace')

891

In [7]:
from langchain_community.utilities import SQLDatabase

In [8]:
db = SQLDatabase.from_uri("sqlite:///titanic.db", sample_rows_in_table_info = 3)
print(db.table_info)


CREATE TABLE "Passenger Data" (
	"index" INTEGER, 
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from Passenger Data table:
index	PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
0	1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
1	2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
2	3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/


## Initialize Language Model from OpenAI

In [9]:
from langchain_openai import ChatOpenAI
import os

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo", 
                 temperature=0, 
                 openai_api_key=os.environ.get("OPENAI_API_KEY"))

In [11]:
llm.config_schema

<bound method Runnable.config_schema of ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000223C8AC1EB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000223C8AD5400>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')>

## Using SQLDatabase Chain

In [12]:
from langchain_experimental.sql.base import SQLDatabaseChain

In [13]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)

In [14]:
sql_db_chain.invoke("How many passengers were there?")



> Entering new SQLDatabaseChain chain...
How many passengers were there?
SQLQuery:SELECT COUNT("PassengerId") AS TotalPassengers
FROM "Passenger Data"
SQLResult: [(891,)]
Answer:Total number of passengers: 891
> Finished chain.


{'query': 'How many passengers were there?',
 'result': 'Total number of passengers: 891'}

In [15]:
# sql_db_chain.invoke("How does survival rate varies on the basis of passenger classes?")

## Custom Prompt

In [17]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)
sql_db_chain.invoke("Provide the gender-wise survival rate along with age bins of 20.")



> Entering new SQLDatabaseChain chain...
Provide the gender-wise survival rate along with age bins of 20.
SQLQuery:SELECT "Sex", 
    CASE 
        WHEN "Age" < 20 THEN '0-19'
        WHEN "Age" >= 20 AND "Age" < 40 THEN '20-39'
        WHEN "Age" >= 40 AND "Age" < 60 THEN '40-59'
        ELSE '60+'
    END AS "Age_Bin",
    AVG("Survived") AS "Survival_Rate"
FROM "Passenger Data"
GROUP BY "Sex", "Age_Bin"
ORDER BY "Sex", "Age_Bin"
LIMIT 5;
SQLResult: [('female', '0-19', 0.7066666666666667), ('female', '20-39', 0.7727272727272727), ('female', '40-59', 0.76), ('female', '60+', 0.7017543859649122), ('male', '0-19', 0.29213483146067415)]
Answer:The gender-wise survival rate along with age bins of 20 are as follows:
- For females aged 0-19, the survival rate is approximately 70.67%
- For females aged 20-39, the survival rate is approximately 77.27%
- For females aged 40-59, the survival rate is 76.00%
- For females aged 60 and above, the survival rate is approximately 70.18%
- For males 

{'query': 'Provide the gender-wise survival rate along with age bins of 20.',
 'result': 'The gender-wise survival rate along with age bins of 20 are as follows:\n- For females aged 0-19, the survival rate is approximately 70.67%\n- For females aged 20-39, the survival rate is approximately 77.27%\n- For females aged 40-59, the survival rate is 76.00%\n- For females aged 60 and above, the survival rate is approximately 70.18%\n- For males aged 0-19, the survival rate is approximately 29.21%'}

In [18]:
connection = connect("titanic.db")

In [19]:
curr = connection.execute('''
SELECT "Sex", 
    CASE 
        WHEN "Age" < 20 THEN '0-19'
        WHEN "Age" >= 20 AND "Age" < 40 THEN '20-39'
        WHEN "Age" >= 40 AND "Age" < 60 THEN '40-59'
        ELSE '60+'
    END AS "Age_Bin",
    AVG("Survived") AS "Survival_Rate"
FROM "Passenger Data"
GROUP BY "Sex", "Age_Bin"
ORDER BY "Sex", "Age_Bin";
''')

In [20]:
print(curr.fetchall())

[('female', '0-19', 0.7066666666666667), ('female', '20-39', 0.7727272727272727), ('female', '40-59', 0.76), ('female', '60+', 0.7017543859649122), ('male', '0-19', 0.29213483146067415), ('male', '20-39', 0.18823529411764706), ('male', '40-59', 0.1839080459770115), ('male', '60+', 0.13013698630136986)]


In [21]:
curr.close()
connection.close()

In [22]:
prompt_template = '''
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
CREATE TABLE "Passenger Data" (
	"index" INTEGER, 
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from Passenger Data table:
index	PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
0	1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
1	2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
2	3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/

Question: {input}
'''

In [23]:
from langchain_core.prompts import PromptTemplate
PROMPT = PromptTemplate.from_template(prompt_template, variables = ['input'])

In [24]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True, prompt = PROMPT)

In [26]:
sql_db_chain.invoke("Provide the gender-wise survival rate along with age bins of 20")



> Entering new SQLDatabaseChain chain...
Provide the gender-wise survival rate along with age bins of 20
SQLQuery:SELECT "Sex", 
       CASE 
           WHEN "Age" < 20 THEN '0-19'
           WHEN "Age" >= 20 AND "Age" < 40 THEN '20-39'
           WHEN "Age" >= 40 AND "Age" < 60 THEN '40-59'
           ELSE '60+'
       END AS "AgeBin",
       AVG("Survived") AS "SurvivalRate"
FROM "Passenger Data"
GROUP BY "Sex", "AgeBin"
ORDER BY "Sex", "AgeBin"
LIMIT 10;
SQLResult: [('female', '0-19', 0.7066666666666667), ('female', '20-39', 0.7727272727272727), ('female', '40-59', 0.76), ('female', '60+', 0.7017543859649122), ('male', '0-19', 0.29213483146067415), ('male', '20-39', 0.18823529411764706), ('male', '40-59', 0.1839080459770115), ('male', '60+', 0.13013698630136986)]
Answer:Female passengers had a higher survival rate compared to male passengers across different age bins. The survival rate for females ranged from approximately 70% to 77%, while for males it ranged from around 13% to 2

{'query': 'Provide the gender-wise survival rate along with age bins of 20',
 'result': 'Female passengers had a higher survival rate compared to male passengers across different age bins. The survival rate for females ranged from approximately 70% to 77%, while for males it ranged from around 13% to 29%.'}

## Create a SQL Query Chain

In [27]:
from langchain.chains import create_sql_query_chain

In [28]:
sql_chain = create_sql_query_chain(llm, db)

In [29]:
sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below.

In [30]:
response = sql_chain.invoke({"question": "How many passengers were there?"})
response

'SELECT COUNT("PassengerId") AS total_passengers\nFROM "Passenger Data"'

## Run SQL Query on your database

In [31]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [32]:
db_execution = QuerySQLDataBaseTool(db = db)
execution_chain = sql_chain | db_execution

In [33]:
response = execution_chain.invoke({"question": "How many passengers were there?"})
response

'[(891,)]'

## Summarize the final result

In [34]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [35]:
from langchain_core.prompts import PromptTemplate

template = '''
Given the following user's quesion, corresponsing SQL query and SQL result, answer the user's question.
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:
'''
prompt = PromptTemplate.from_template(template)

In [36]:
output = prompt | llm | StrOutputParser()
chain = (RunnablePassthrough.assign(query = sql_chain).assign(result = itemgetter("query") | db_execution) | output)

In [37]:
ch = RunnablePassthrough.assign(query = sql_chain)

In [38]:
ch.invoke({"question": "How does the prices varies with respect to passenger classes?"})

{'question': 'How does the prices varies with respect to passenger classes?',
 'query': 'SELECT "Pclass", AVG("Fare") AS "Average Fare"\nFROM "Passenger Data"\nGROUP BY "Pclass"\nORDER BY "Pclass";\nLIMIT 5;'}

In [39]:
ch2 = RunnablePassthrough.assign(query = sql_chain).assign(result = itemgetter("query") | db_execution)

In [40]:
ch2.invoke({"question": "How does the prices varies with respect to passenger classes?"})

{'question': 'How does the prices varies with respect to passenger classes?',
 'query': 'SELECT "Pclass", AVG("Fare") AS "Average Fare"\nFROM "Passenger Data"\nGROUP BY "Pclass"\nORDER BY "Pclass" ASC\nLIMIT 5;',
 'result': '[(1, 84.15468749999992), (2, 20.66218315217391), (3, 13.675550101832997)]'}

In [42]:
chain.invoke({"question": "How does the prices varies with respect to passenger classes?"})

'The prices vary significantly with respect to passenger classes. On average, passengers in first class (Pclass 1) paid the highest fare of $84.15, followed by passengers in second class (Pclass 2) with an average fare of $20.66, and passengers in third class (Pclass 3) with the lowest average fare of $13.68. This shows that there is a clear difference in pricing based on the passenger class.'

## Showcasing in UI using Gradio

In [43]:
import gradio as gr

In [44]:
template = '''
Given the following user's quesion, corresponsing SQL query and SQL result, answer the user's question.
Question: {question}
SQL Query: {sql_query}
SQL Result: {result}
Answer:
'''
prompt = PromptTemplate.from_template(template)

In [45]:
def create_chain(question):
    db = SQLDatabase.from_uri("sqlite:///titanic.db", sample_rows_in_table_info = 3)
    sql_chain = create_sql_query_chain(llm, db)
    db_execution = QuerySQLDataBaseTool(db = db)
    output = prompt | llm | StrOutputParser()
    chain = (RunnablePassthrough.assign(sql_query = sql_chain).assign(result = itemgetter("sql_query") | db_execution) | output)

    return chain.invoke({"question": question})


In [46]:
def extract_data(user_message, history):
    question_with_history = ""
    for hist in history[-3:]:
        question_with_history += f"User: {hist[0]}\nAssistant: {hist[1]}\n"
    question_with_history += f"User: {user_message}\n"
    print("Input to LLM:\n", question_with_history)

    bot_message = create_chain(question_with_history)

    history += [[user_message, bot_message]]

    return bot_message, history

In [47]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label = "Chat with Data")
    msg = gr.Textbox(label = "Question", placeholder = "Enter your question here")
    clear = gr.Button("Clear")

    def user(user_message, history):
        bot_message, history = extract_data(user_message, history)
        print("LLM Response: ", bot_message)
        return "", history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Talk with Data

In [48]:
from transformers import AutoProcessor, SeamlessM4TModel
import torchaudio
import time

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")

C:\Users\atulk\.conda\envs\hgface\lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
def speech_to_text(filepath):
    t1 = time.time()
    audio, orig_freq =  torchaudio.load(filepath, backend = "soundfile")
    audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
    audio_inputs = processor(audios=audio, return_tensors="pt")
    output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech = False)
    generated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens = True)

    print("Time taken for speech to text: ", time.time() - t1)
    print("Speech to text output: ", generated_text_from_audio)

    return generated_text_from_audio

In [50]:
import subprocess
def text_to_speech(text):
    subprocess.call(['edge-tts', '--text', text, '--write-media', 'output.wav', '--write-subtitles', 'hello.vtt'])

In [51]:
def create_chain(question):
    db = SQLDatabase.from_uri("sqlite:///titanic.db", sample_rows_in_table_info = 3)
    sql_chain = create_sql_query_chain(llm, db)
    db_execution = QuerySQLDataBaseTool(db = db)
    output = prompt | llm | StrOutputParser()
    chain = (RunnablePassthrough.assign(sql_query = sql_chain).assign(result = itemgetter("sql_query") | db_execution) | output)

    return chain.invoke({"question": question})

def extract_data(user_message, history):
    question_with_history = ""
    for hist in history[-3:]:
        question_with_history += f"User: {hist[0]}\nAssistant: {hist[1]}\n"
    question_with_history += f"User: {user_message}\n"
    print("Input to LLM:\n", question_with_history)

    bot_message = create_chain(question_with_history)

    history += [[user_message, bot_message]]

    return bot_message, history

In [52]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label = "Chat with Data")
    output_audio = gr.Audio(label = "Output", autoplay = True)
    with gr.Row():
        msg = gr.Textbox(label = "Question", placeholder = "Enter your question here")
        audio = gr.Audio(source = "microphone", type = "filepath", label = "Record")
    with gr.Row():
        submit = gr.Button("Submit")
        clear = gr.ClearButton([chatbot, audio, output_audio])

    def user(user_message, history, filepath):
        if not user_message and filepath:
            user_message = speech_to_text(filepath)
        t1 = time.time()
        bot_message, history = extract_data(user_message, history)
        text_to_speech(bot_message)

        print("LLM Response: ", bot_message)
        print("Time taken for llm generation: ", time.time() - t1)
        return "", history, "output.wav"

    msg.submit(user, [msg, chatbot, audio], [msg, chatbot, output_audio], queue=False)
    submit.click(user, [msg, chatbot, audio], [msg, chatbot, output_audio], queue=False)
    
demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
